In [1]:
import numpy as np
import torch
import pandas as pd
from torchvision import models
from torchvision.models.densenet import densenet161
import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score
import ISBI_data
import ISBI_rareset
from metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from nets import ResNet, ResNext, ViT, ResNet152, Densenet161, effNetB7, effNetB6

def loadModel(ckpt_path):
    model = ResNext()
    ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
    new_dict = {k.replace('vit.', 'model.'): v for k, v in ckpt['state_dict'].items()}
    model.load_state_dict(new_dict)
    model.eval()
    model.cuda()
    return model

########## load models ###########
model0 = loadModel('saved_model/ISBI-WeightedBCE-ResNext101-epoch=013-val_loss=0.0892.ckpt')
model1 = loadModel('data/checkpoints/ISBI-WeightedBCE-boosting-b1-ResNext101-b0-epoch=009-val_loss=0.0870.ckpt')
model2 = loadModel('data/checkpoints/ISBI-WeightedBCE-boosting-b1-ResNext101-b1-epoch=008-val_loss=0.0862.ckpt')
model3 = loadModel('data/checkpoints/ISBI-WeightedBCE-boosting-b1-ResNext101-b2-epoch=011-val_loss=0.0885.ckpt')
model4 = loadModel('data/checkpoints/ISBI-WeightedBCE-boosting-b1-ResNext101-b3-epoch=010-val_loss=0.0870.ckpt')
model5 = loadModel('data/checkpoints/ISBI-WeightedBCE-boosting-b1-ResNext101-b4-epoch=010-val_loss=0.0832.ckpt')
M = []
M.append(model0)
M.append(model1)
M.append(model2)
M.append(model3)
M.append(model4)
M.append(model5)

/usr/mvl2/llzqd/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


In [7]:
##################### load train dataset for RF training #################
train_img_path = '../Training_Set/Training/'
train_df = '../Training_Set/RFMiD_Training_Labels.csv'
train_set = ISBI_data.ISBIDataset(train_df, train_img_path, testing=True, reweight=False)
N = len(train_set)
batch_size = 32
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=False, 
                        num_workers=24)

outs_train = np.zeros((N, 29, len(M)))
labels_train = np.zeros((N, 29))
for n in range(len(M)):
    for i, (imgs, label) in enumerate(tqdm.tqdm(train_dataloader)):

        idx = i * batch_size
        imgs = imgs.cuda()
        out = M[n](imgs).detach().cpu().numpy()
        #out = np.round(out).astype('int').clip(1, None)
        outs_train[idx:idx + len(out),:, n] = out
        if n ==0:
            labels_train[idx:idx + len(label),:]  = label.detach().cpu().numpy()
outs_train = torch.tensor(outs_train).numpy()  
##################### load valid dataset for RF training #################
valid_img_path = '../Evaluation_Set/Validation/'
valid_df = '../Evaluation_Set/RFMiD_Validation_Labels.csv'
valset = ISBI_data.ISBIDataset(valid_df, valid_img_path, testing=True, reweight=False)
N = len(valset)
batch_size = 32
valid_dataloader = DataLoader(valset, batch_size=batch_size, shuffle=False, 
                        num_workers=24)

outs_valid = np.zeros((N, 29, len(M)))
labels_valid = np.zeros((N, 29))
for n in range(len(M)):
    for i, (imgs, label) in enumerate(tqdm.tqdm(valid_dataloader)):

        idx = i * batch_size
        imgs = imgs.cuda()
        out = M[n](imgs).detach().cpu().numpy()
        #out = np.round(out).astype('int').clip(1, None)
        outs_valid[idx:idx + len(out),:, n] = out
        if n ==0:
            labels_valid[idx:idx + len(label),:]  = label.detach().cpu().numpy()
outs_valid = torch.tensor(outs_valid).numpy()  

100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


In [19]:
sig = torch.nn.Sigmoid()
outs_train = sig(torch.tensor(outs_train)).numpy() 
outs_valid = sig(torch.tensor(outs_valid)).numpy()  
outs = np.vstack((outs_train, outs_valid))
labels = np.vstack((labels_train, labels_valid))

In [37]:
import joblib
from joblib import dump
from sklearn.linear_model import LogisticRegression
for i in range(labels.shape[1]):
    reg = LogisticRegression(max_iter=2000)
    reg.fit(outs_valid.reshape((len(outs_valid),-1)), labels_valid[:,i])
    dump(reg, f'ensemble/LogisticR{i}.joblib')

In [14]:
##################### load test dataset for RF training #################
testing_img_path = '../Test_Set/Test/'
testing_df = '../Test_Set/RFMiD_Testing_Labels.csv'
testset = ISBI_data.ISBIDataset(testing_df, testing_img_path, testing=True, reweight=False)
N = len(testset)
batch_size = 32
test_dataloader = DataLoader(testset, batch_size=batch_size, shuffle=False, 
                        num_workers=24)

outs_test = np.zeros((N, 29, len(M)))
labels_test = np.zeros((N, 29))
for n in range(len(M)):
    for i, (imgs, label) in enumerate(tqdm.tqdm(test_dataloader)):

        idx = i * batch_size
        imgs = imgs.cuda()
        out = M[n](imgs).detach().cpu().numpy()
        outs_test[idx:idx + len(out),:, n] = out
        if n ==0:
            labels_test[idx:idx + len(label),:]  = label.detach().cpu().numpy()

/usr/mvl2/llzqd/ISBI/code/ISBI_data.py:143: RuntimeWarning: divide by zero encountered in double_scalars
  w[i] = np.sum(c)/c[i]
100%|██████████| 20/20 [00:17<00:00,  1.17it/s]


In [34]:
regressor_predoct = np.zeros_like(labels_test)
for i in range(labels_test.shape[1]):
    reg = joblib.load(f'ensemble/regressor{i}.joblib')
    regressor_predoct[:,i] = reg.predict(outs_test[:,i,:].reshape((len(labels_test),-1)))

In [38]:
outs_test = sig(torch.tensor(outs_test)).numpy()  
regressor_predoct = np.zeros_like(labels_test)
for i in range(labels_test.shape[1]):
    reg = joblib.load(f'ensemble/LogisticR{i}.joblib')
    regressor_predoct[:,i] = reg.predict_proba(outs_test.reshape((len(labels_test),-1)))[:,1]

In [39]:
auc1 = roc_auc_score(labels_test[:,0], regressor_predoct[:,0])
print(f'AUC of Challenge 1: {auc1}')

auc2 = roc_auc_score(labels_test[:,1:], regressor_predoct[:,1:])
print(f'AUC of Challenge 2: {auc2}')

mAP = average_precision_score(labels_test[:,1:], regressor_predoct[:,1:])
print(f'mAP of Challenge 2: {mAP}')

C1_Score = auc1
C2_Score = mAP * 0.5 + auc2 * 0.5
final_Score =  C2_Score * 0.5 + C1_Score * 0.5
print(f'C1 Score: {C1_Score} C2 Score: {C2_Score} Final Score: {final_Score}')

AUC of Challenge 1: 0.9805763671759778
AUC of Challenge 2: 0.8694631869061098
mAP of Challenge 2: 0.4760738071482817
C1 Score: 0.9805763671759778 C2 Score: 0.6727684970271958 Final Score: 0.8266724321015868


In [6]:
import pretrainedmodels
print(pretrainedmodels.model_names)
print(pretrainedmodels.pretrained_settings['se_resnext50_32x4d'])

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']
{'imagenet': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth', 'input_space': 'RGB', 'input_size': [3, 224, 224], 'input_range': [0, 1], 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'num_classes': 1000}}


In [7]:
model = pretrainedmodels.__dict__['se_resnext50_32x4d'](num_classes=1000, pretrained='imagenet')
model

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /usr/mvl2/llzqd/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


  0%|          | 0.00/105M [00:00<?, ?B/s]

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [8]:
from torchvision import models
model2 = models.resnext101_32x8d(pretrained=True)
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1